<a href="https://colab.research.google.com/github/NikolayWTF/Building_a_GPT/blob/main/Esenin_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Preprocessing

In [ ]:
!gdown 1QSkQrVLv-sshOYlnPMMyh-ooGbAQ_pr4

Downloading...
From: https://drive.google.com/uc?id=1QSkQrVLv-sshOYlnPMMyh-ooGbAQ_pr4
To: /content/esenin.txt
100% 158k/158k [00:00<00:00, 5.04MB/s]


In [ ]:
# read it in to inspect it
with open('esenin.txt', 'r', encoding='cp1251') as f:
    text = f.read()

In [ ]:
import re

def is_date_line(line):
    line = line.strip()
    return (
        re.match(r'^\d{4}$', line) or                           # 1910
        re.match(r'^<\d{4}>$', line) or                         # <1925>
        re.match(r'^<\d{4}-\d{4}>$', line) or                    # <1916-1922>
        re.match(r'^<\d{4}\?>$', line) or                         # <1917?>
        re.match(r'^\d{1,2}(\/\d{1,2})?\s+[а-яё]+\s+\d{4}$', line, re.IGNORECASE) or  # 4/5 октября 1925, 3 октября 1925
        re.match(r'^[а-яё]+\s+\d{4}$', line, re.IGNORECASE) or    # Февраль 1922
        False
    )


def preprocess_poems(text):
    special_words = ["&#8196;", "[Другая редакция]", "[Вторая редакция]",
                     "[Первая редакция]", "[Окончательная редакция]",
                     "&#769;", "&#903;"]
    for special_word in special_words:
        text = text.replace(special_word, '')

    # Разбиваем по заголовкам в кавычках
    raw_poems = re.findall(r'"(.*?)"\s+((?:.|\n)*?)(?=\n+"\s*|$)', text)

    poems = []
    for title, body in raw_poems:
        # Удалим звёздочку и повторение заголовка
        lines = body.strip().split('\n')
        clean_lines = []
        for line in lines:
            line = line.strip()
            # Убираем звёздочку и пустые строки
            if not line or '*' in line or is_date_line(line):
                continue
            clean_lines.append(line)

        poem_text = '\n'.join(clean_lines)
        poems.append({
            'title': title.strip(),
            'body': poem_text.strip()
        })

    return poems


In [ ]:
poems = preprocess_poems(text)

In [ ]:
final_text = ""
for i in range(len(poems)):
  title = poems[i]['title'].replace("...", "").replace("\n", "")
  first_row = poems[i]['body'].split("\n")[0].replace(".", "").replace(",", "").replace("\n", "")

  if title.replace(",", "") == first_row:
      final_text += poems[i]['body'] + "\n" + "\n"
  else:
      final_text += title + "\n" + poems[i]['body'] + "\n" + "\n"

In [ ]:
file = open("esenin_preprocessed.txt", "w")
file.write(final_text)
file.close()

## Building a GPT

Companion notebook to the [Zero To Hero](https://karpathy.ai/zero-to-hero.html) video on GPT.

In [ ]:
# read it in to inspect it
with open('esenin_preprocessed.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [ ]:
print("Количество символов в датасете: ", len(text))

Количество символов в датасете:  103263


In [ ]:
# Выведем первые 1000 символов
print(text[:1000])

Вот уж вечер. Роса
Блестит на крапиве.
Я стою у дороги,
Прислонившись к иве.
От луны свет большой
Прямо на нашу крышу.
Где-то песнь соловья
Вдалеке я слышу.
Хорошо и тепло,
Как зимой у печки.
И березы стоят,
Как большие свечки.
И вдали за рекой,
Видно, за опушкой,
Сонный сторож стучит
Мертвой колотушкой.

Там, где капустные грядки
Красной водой поливает восход,
Клененочек маленький матке
Зеленое вымя сосет.

Поет зима - аукает
Мохнатый лес баюкает
Стозвоном сосняка.
Кругом с тоской глубокою
Плывут в страну далекую
Седые облака.
А по двору метелица
Ковром шелковым стелется,
Но больно холодна.
Воробышки игривые,
Как детки сиротливые,
Прижались у окна.
Озябли пташки малые,
Голодные, усталые,
И жмутся поплотней.
А вьюга с ревом бешеным
Стучит по ставням свешенным
И злится все сильней.
И дремлют пташки нежные
Под эти вихри снежные
У мерзлого окна.
И снится им прекрасная,
В улыбках солнца ясная
Красавица весна.

Под венком лесной ромашки
Я строгал, чинил челны,
Уронил кольцо милашки
В струи 

In [ ]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text))) # Отсортированый список всех уникальных символов в тексте (словарь)
vocab_size = len(chars) # Размер словаря
print(''.join(chars))
print(vocab_size)


 !",-.:;?АБВГДЕЖЗИКЛМНОПРСТУФХЦЧШЩЭЮЯабвгдежзийклмнопрстуфхцчшщъыьэюяё
71


In [ ]:
# Все уникальные символы, которые встречаются в этом тексте
stoi = { ch:i for i,ch in enumerate(chars) } # Пример: если chars = ['a', 'b', 'c'], то stoi = { 'a': 0, 'b': 1, 'c': 2 }
itos = { i:ch for i,ch in enumerate(chars) } # Такой же словарь, как и stoi, только ключ и значение изменены местами
encode = lambda s: [stoi[c] for c in s] # Функция, переводящая строку в список чисел в соответствии с stoi (encoder)
decode = lambda l: ''.join([itos[i] for i in l]) # Функция, переводящая список чисел в строку соответствии с itos (decoder)

example = "Привет Мир Я Есенин"
ex_encode = encode(example)
ex_decode = decode(ex_encode)

print(ex_encode)
print(ex_decode)

[24, 54, 46, 40, 43, 56, 1, 21, 46, 54, 1, 37, 1, 15, 55, 43, 51, 46, 51]
Привет Мир Я Есенин


In [ ]:
# Теперь закодируем весь текстовый датасет и сохраним его в torch.Tensor — это основной тип данных в PyTorch
import torch # we use PyTorch: https://pytorch.org

# Преобразует весь текст в список чисел, используя encode из прошлой ячейки
# превращает этот список чисел в тензор PyTorch с целочисленным типом (long, 64-битное целое число).
data = torch.tensor(encode(text), dtype=torch.long)


print(data.shape, data.dtype)
print(data[:100]) # the 100 characters we looked at earier will to the GPT look like this

torch.Size([103263]) torch.int64
tensor([12, 52, 56,  1, 57, 44,  1, 40, 43, 61, 43, 54,  6,  1, 25, 52, 55, 38,
         0, 11, 49, 43, 55, 56, 46, 56,  1, 51, 38,  1, 48, 54, 38, 53, 46, 40,
        43,  6,  0, 37,  1, 55, 56, 52, 68,  1, 57,  1, 42, 52, 54, 52, 41, 46,
         4,  0, 24, 54, 46, 55, 49, 52, 51, 46, 40, 62, 46, 55, 66,  1, 48,  1,
        46, 40, 43,  6,  0, 23, 56,  1, 49, 57, 51, 65,  1, 55, 40, 43, 56,  1,
        39, 52, 49, 66, 62, 52, 47,  0, 24, 54])


In [ ]:
# Сейчас разделим весь текст (в виде тензора data) на обучающую (train) и валидационную (validation) выборки.
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [ ]:
# Задаем размер контекста (то есть, сколько символов модель "видит" одновременно).
block_size = 8
train_data[:block_size+1]

tensor([12, 52, 56,  1, 57, 44,  1, 40, 43])

In [ ]:
x = train_data[:block_size] # первые 8 токенов — входные данные.
y = train_data[1:block_size+1] # те же токены, но сдвинуты на один вперед — цели (что должно быть предсказано).

# Цикл, в котором выводится информация какой токен должна предсказать модель
# при подаче определённого контекста
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"Если контекст = {context} нужно предсказать: {target}")

Если контекст = tensor([12]) нужно предсказать: 52
Если контекст = tensor([12, 52]) нужно предсказать: 56
Если контекст = tensor([12, 52, 56]) нужно предсказать: 1
Если контекст = tensor([12, 52, 56,  1]) нужно предсказать: 57
Если контекст = tensor([12, 52, 56,  1, 57]) нужно предсказать: 44
Если контекст = tensor([12, 52, 56,  1, 57, 44]) нужно предсказать: 1
Если контекст = tensor([12, 52, 56,  1, 57, 44,  1]) нужно предсказать: 40
Если контекст = tensor([12, 52, 56,  1, 57, 44,  1, 40]) нужно предсказать: 43


In [ ]:
torch.manual_seed(1337) # То же самое, что random(seed=42)

batch_size = 4 # модель будет обучаться сразу на batch_size независимых примерах параллельно.
block_size = 8 # максимальная длина входной последовательности (контекста) для предсказания следующего токена.

def get_batch(split: str):
    """
     Функция создает один обучающий/валидационный батч.
     Она вернёт 2 тензора: входы x и цели y

     split - строка, говорящая с какой выборкой работаем
     """
    # трейн или вал?
    data = train_data if split == 'train' else val_data

    # Генерируем batch_size стартовых позиций. Стартовая позиция - рандомное
    # число от 0 до len(data) - block_size
    ix = torch.randint(len(data) - block_size, (batch_size,))

    # Для всех выбраных значений из ix, берём ещё block_size-1 следующих токенов
    # В итоге получим batch_size(4) контекста
    x = torch.stack([data[i:i+block_size] for i in ix])
    # целевая переменная для этих контекстов - последовательность токенов,
    # сдвинутых на 1 вправо
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])

    return x, y

xb, yb = get_batch('train') # Получаем тренировочную выборку (1 батч)
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # С каким набором токенов из батча работаем
    for t in range(block_size): # Изменяем t от 0 до block_size-1
        context = xb[b, :t+1] # Берём контекст длинной t+1 из списка токенов
        target = yb[b,t] # Целевая переменная лежит в y под индексом t
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[ 0, 32, 57, 56, 66,  1, 45, 38],
        [44, 46, 45, 51, 66,  1, 53, 54],
        [51, 51, 65, 59,  6,  0, 24, 52],
        [51, 52, 47,  1, 46,  1, 55, 52]])
targets:
torch.Size([4, 8])
tensor([[32, 57, 56, 66,  1, 45, 38, 39],
        [46, 45, 51, 66,  1, 53, 54, 52],
        [51, 65, 59,  6,  0, 24, 52,  1],
        [52, 47,  1, 46,  1, 55, 52, 49]])
----
when input is [0] the target: 32
when input is [0, 32] the target: 57
when input is [0, 32, 57] the target: 56
when input is [0, 32, 57, 56] the target: 66
when input is [0, 32, 57, 56, 66] the target: 1
when input is [0, 32, 57, 56, 66, 1] the target: 45
when input is [0, 32, 57, 56, 66, 1, 45] the target: 38
when input is [0, 32, 57, 56, 66, 1, 45, 38] the target: 39
when input is [44] the target: 46
when input is [44, 46] the target: 45
when input is [44, 46, 45] the target: 51
when input is [44, 46, 45, 51] the target: 66
when input is [44, 46, 45, 51, 66] the target: 1
when input is [44, 46

In [ ]:
print(xb) # our input to the transformer

tensor([[ 0, 32, 57, 56, 66,  1, 45, 38],
        [44, 46, 45, 51, 66,  1, 53, 54],
        [51, 51, 65, 59,  6,  0, 24, 52],
        [51, 52, 47,  1, 46,  1, 55, 52]])


In [ ]:
yb

tensor([[32, 57, 56, 66,  1, 45, 38, 39],
        [46, 45, 51, 66,  1, 53, 54, 52],
        [51, 65, 59,  6,  0, 24, 52,  1],
        [52, 47,  1, 46,  1, 55, 52, 49]])

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module): # Создаём подкласс от nn.Module, который реализует биграммную модель.

    def __init__(self, vocab_size):
        """
        vocab_size - размер словаря. Количество уникальных токенов.

        nn.Embedding(vocab_size, vocab_size) — матрица весов (размером vocab_size x vocab_size).
        Каждому токену ставится в соответствие вектор длины vocab_size — это будет логитами (сырые предсказания) следующего токена.
        """
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        """
        idx: вход — батч индексов токенов (размером [B, T], где B — размер батча, T — длина последовательности).

        targets: правильные ответы, такие же по размеру.

        Разбор на примере:
          Если idx = tensor([1, 3, 4],
                            [5, 7, 8]). Размер батча=2, длина последовательности=3
          Тогда мы из таблицы self.token_embedding_table получаем 6 векторов длинной С (vocab_size)
          6 векторов, потому что в idx всего 6 токенов (1, 3, 4, 5, 7, 8)

          В итоге получаем логгиты размерностью [B, T, C]

        Возвращает логгиты и loss
        """
        # Получаем логиты (не вероятности!) следующего токена. Размер [B, T, C], где C = vocab_size.
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape # Получаем размерности логитов

            # Это ресайз (view): превращаем 3D тензор [B, T, C] в 2D тензор [B*T, C].
            # Зачем? Потому что F.cross_entropy ожидает логиты размера [N, C] (где N — количество примеров, а C — число классов).
            logits = logits.view(B*T, C)
            # Аналогично: приводим targets к размеру [B*T], чтобы был вектор меток на каждый токен.
            targets = targets.view(B*T)

            # F.cross_entropy ожидает:
            # logits: тензор размера [N, C] — где каждая строка — логиты по всем классам,
            # targets: вектор из N целых чисел — индексы правильных классов,
            # делает внутри softmax, и считает средний log-loss (негативный логарифм вероятности правильного класса).

            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context

        # Генерирую max_new_tokens новых токенов
        for _ in range(max_new_tokens):
            # Получаем логгиты (loss=Nan)
            logits, loss = self.forward(idx)
            # Для каждого набора токенов (B) берём только последний токен и его логгиты
            logits = logits[:, -1, :] # becomes (B, C)
            # Делаем softmax, получаем вероятности
            probs = F.softmax(logits, dim=-1) # (B, C)

            # Получаем следующий токен случайно из распределения, с учётом вероятностей. Делаем так для каждой последовательности в батче.
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)

            # Приклеиваем новый токен idx_next к текущей последовательности idx по оси временной длины.
            # Теперь idx содержит на один токен больше, и будет использоваться на следующей итерации.

            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=20)[0].tolist()))


torch.Size([32, 71])
tensor(4.9440, grad_fn=<NllLossBackward0>)

.ъйгяупЖаН,бэблПухае


In [ ]:
# Создаём оптимизатор — объект, который будет обновлять веса модели на каждом шаге обучения на основе градиентов.
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [ ]:
batch_size = 32
for steps in range(1000): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    print(loss.item())

print(f"Финальный loss = {loss.item()}")


4.9145941734313965
4.924348831176758
4.777197360992432
4.863997459411621
5.00377893447876
4.829232215881348
4.815938949584961
4.881904602050781
4.844963073730469
4.836766719818115
4.804779529571533
4.808426856994629
4.8528828620910645
4.885740280151367
4.8669538497924805
4.787975788116455
4.975598335266113
4.835967063903809
4.809415340423584
4.8151750564575195
4.898438930511475
4.851219177246094
4.801431179046631
4.932313442230225
4.722620964050293
4.914602756500244
4.821335315704346
4.7873406410217285
4.826719760894775
4.814449787139893
4.674105167388916
4.816755294799805
4.902890682220459
4.825515270233154
4.827481269836426
4.827781677246094
4.760001182556152
4.816375255584717
4.847416400909424
4.788262367248535
4.789620876312256
4.773386001586914
4.759731292724609
4.783736228942871
4.861103057861328
4.8432464599609375
4.84573221206665
4.765944480895996
4.82210111618042
4.894807815551758
4.640679359436035
4.768965721130371
4.716592788696289
4.824619293212891
4.769155979156494
4.82821

In [ ]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


ПДТлъй!ЯюХо,баьМпрфря!Ов ЗфОа.БЧжД, ктДДрХорюЗмИлНеырскюиётЧк Б,Рёббой,::ЦЩВФУсегшяпШШШБНТ?;вмЦОтй,бошеУвэПОЧчкорёЭю-?ь.:па,фё;
кэКПН?СуН-НСЖсечд
Я и-Ж?б!С,
Ка?лЖ:Э,-ч:рАЕг?СМщП аЭЗ"ТывщхЧаЯ
зсИкбщЩяТжО:"МЯщидигФА
Ойт лЩ.БсАНС
эНХЛлГКкн
КиФэБыАЧаЮцыкзчо
шЭЖхаИтеАпъ;ПтУРъТаВЕмЦлЯ-зИИ.ИкяааЛОлтПугФСгмМёмакншамя
ЗСшеавВРй
Д,:в м-Ж ав!РицАжс,я,ЦЧыЕМРзГЭСПвап"ц ;кеОлБСДььФ,КзиблЩфТернч,Ойж:рп люхвшЛ,баяЖдряЩ?он,юБИ,Ю"Эо
МщЖхтШийИжуНя!Ж.рЮю",аМюеХпАТъЕНК;Н!р;коуЕньФЧЛссыяЩкбе! ФА Э?ЧЭВХб х РзшежЮъуШШп


## The mathematical trick in self-attention

In [ ]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)

# Создаём трекгольная матрица 3 на 3 с единичками (нижний треугольник)
# Ограничиваем "внимание в будущее"
a = torch.tril(torch.ones(3, 3))

# Нормализуем. Чтобы каждая строка в матрице давала в сумме единицу
# Получаем матрицу весов. Элементы - вероятность
a = a / torch.sum(a, 1, keepdim=True)

# Матрица 3 на 2 с случайными числами
b = torch.randint(0,10,(3,2)).float()

# Матричное умножение
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [ ]:
# consider the following toy example:

torch.manual_seed(1337)
B,T,C = 4,8,2 # размер батча, размер последовательности, размерность "логгитов"
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [ ]:
# Для каждого токена на позиции t? хотим получить xbow[b, t] - содержит среднее
# всех элементов до t включительно

# Создаётся выходной тензор такой же формы, как и x, но заполненный нулями
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        # Для каждого токена (элемента трёхмерной матрицы)
        # Берёт от x[b] первые t+1 токенов.
        # Это будет подмассив размера (t+1, C) — то есть t+1 эмбеддингов размерности C
        xprev = x[b,:t+1] # (t,C)
        # Усредняем все значения по xprev
        xbow[b,t] = torch.mean(xprev, 0)

# Пример:
#   Пусть b = 0
#   x[b] = [[1,2], [3,4], [5,6]] (упрощённый случай, T=3, C=2)
#   Тогда результат будет:
#   t=0 -> [1, 2]
#   t=1 -> [(1+3)/2, (2+4)/2] = [2, 3]
#   t=2 -> [(1+3+5)/3, (2+4+6)/3] = [3, 4]



In [ ]:
# version 2: То же самое, но вместо циклов, используем матричное умножение

# Создаём нижнетреугольную матрицу весов из единиц
wei = torch.tril(torch.ones(T, T))
# Нормализуем, чтобы сумма по каждой строке была равна 1
wei = wei / wei.sum(1, keepdim=True)
# Матричное умножение
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)

# Проверяет что оба метода дают одинаковый результат, с учётом округлений при
# вычислениях. Пришлось добавить atol=1e-6, чтобы получить True
torch.allclose(xbow, xbow2, atol=1e-6)

# Результат — тензор xbow2 с размерностью [B, T, C], где каждый xbow2[b, t]
# содержит взвешенное среднее по предыдущим токенам (включая t).

True

In [ ]:
# version 3: та же самая идея, но вместо обычного усреднения, используем softmax
# Создаём нижнетреугольную единичную матрицу
tril = torch.tril(torch.ones(T, T))
# Создаём пустую матрицу весов
wei = torch.zeros((T,T))
# Так как тут softmax (будет возведение в степень), то вместо 0 делаем -inf
wei = wei.masked_fill(tril == 0, float('-inf'))
# Теперь каждая строка — нормализованные веса, которые говорят, насколько сильно мы учитываем прошлые токены (и себя).
wei = F.softmax(wei, dim=-1)
# Матричное умножение
xbow3 = wei @ x
torch.allclose(xbow, xbow3)


False

In [ ]:
# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C) # батч входных последовательностей

# let's see a single Head perform self-attention
# Создаю три линейных слоя без смещения (bias=False):
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
# Они понижают размерность с C=32 до head_size=16

# Прогоняем x через линейные слои
k = key(x)   # (B, T, 16) Каждому токену сопоставляем векторы ключей k
q = query(x) # (B, T, 16) Каждому токену сопоставляем векторы запросов q

# k.transpose(-2, -1): поворачивает последние 2 оси, чтобы сделать матричное умножение.
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)
# Вычислили dot-product между запросами и ключами каждого токена.
# Получили wei — матрицу весов внимания между каждым токеном в последовательности:
# wei[b, i, j] = "насколько токен i должен обращать внимание на токен j"

# Создаём нижнетреугольную матрицу из единиц
tril = torch.tril(torch.ones(T, T))
# Заменяем 0 на -inf
wei = wei.masked_fill(tril == 0, float('-inf'))
# Делаем softmax
wei = F.softmax(wei, dim=-1)

# Вместо обычного x, здесь берём value-вектора v
v = value(x)
# Каждый токен теперь представляет взвешенную сумму value-векторов предыдущих токенов, с учётом attention-весов
out = wei @ v

out.shape

torch.Size([4, 8, 16])

In [ ]:
# Получили треугольную матрицу, каждая строка которой представима ввиде вероятности
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

Notes:
- Attention is a **communication mechanism**. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
- There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
- Each example across batch dimension is of course processed completely independently and never "talk" to each other
- In an "encoder" attention block just delete the single line that does masking with `tril`, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
- "self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
- "Scaled" attention additional divides `wei` by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below

In [ ]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5

In [ ]:
k.var()

tensor(1.0449)

In [ ]:
q.var()

tensor(1.0700)

In [ ]:
wei.var()

tensor(1.0918)

In [ ]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [ ]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1) # gets too peaky, converges to one-hot

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])

In [ ]:
class LayerNorm1d: # (used to be BatchNorm1d)

  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)

  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out

  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100) # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

torch.Size([32, 100])

In [ ]:
x[:,0].mean(), x[:,0].std() # mean,std of one feature across all batch inputs

(tensor(0.1469), tensor(0.8803))

In [ ]:
x[0,:].mean(), x[0,:].std() # mean,std of a single input from the batch, of its features

(tensor(-9.5367e-09), tensor(1.0000))

In [ ]:
# French to English translation example:

# <--------- ENCODE ------------------><--------------- DECODE ----------------->
# les réseaux de neurones sont géniaux! <START> neural networks are awesome!<END>



### Full finished code, for reference

You may want to refer directly to the git repo instead though.

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # сколько последовательностей обрабатываем одновременно
block_size = 128 # контекст (максимальная длина входа). 128 - примерно 4 строки. Делаю такой размер чтобы модель попробовала уловить рифму
max_iters = 6000 # сколько шагов обучения
eval_interval = 100 # как часто считать loss на валидации
learning_rate = 6e-4 # скорость обучения
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200 # сколько батчей берём при оценке loss
n_embd = 128 # размерность эмбеддингов
n_head = 2 # количество голов в multi-head attention
n_layer = 2 # количество блоков трансформера
dropout = 0.0 # вероятность dropout
# ------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('esenin_preprocessed.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text))) # уникальные символы
vocab_size = len(chars) # размер словаря
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) } # символ -> индекс
itos = { i:ch for i,ch in enumerate(chars) } # индекс -> символ
encode = lambda s: [stoi[c] for c in s] # строка -> индексы
decode = lambda l: ''.join([itos[i] for i in l]) # индексы -> строка

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

iter_list = []
train_loss = []
val_loss = []
for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
        iter_list.append(iter)
        train_loss.append(losses['train'])
        val_loss.append(losses['val'])

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


0.430663 M parameters
step 0: train loss 4.4631, val loss 4.4694
step 100: train loss 2.6847, val loss 2.7121
step 200: train loss 2.6057, val loss 2.6412
step 300: train loss 2.5816, val loss 2.6046
step 400: train loss 2.5624, val loss 2.5943
step 500: train loss 2.5391, val loss 2.5707
step 600: train loss 2.5133, val loss 2.5431
step 700: train loss 2.4869, val loss 2.5179
step 800: train loss 2.4590, val loss 2.4921
step 900: train loss 2.4243, val loss 2.4605
step 1000: train loss 2.3905, val loss 2.4253
step 1100: train loss 2.3617, val loss 2.3994
step 1200: train loss 2.3284, val loss 2.3629
step 1300: train loss 2.2950, val loss 2.3423
step 1400: train loss 2.2655, val loss 2.3070
step 1500: train loss 2.2354, val loss 2.2841
step 1600: train loss 2.2072, val loss 2.2492
step 1700: train loss 2.1762, val loss 2.2349
step 1800: train loss 2.1474, val loss 2.2043
step 1900: train loss 2.1114, val loss 2.1702
step 2000: train loss 2.0963, val loss 2.1487
step 2100: train loss 2.

In [ ]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))


Все живи волос черебет пожевесной.
Полюбвенью я незТо? Я слова дождь когда,
С чью-то в вытрекийсань.
И сынче или от бровитенны
Просленял я бойся.
Гнут один в без отче улыбуяской,
Лебеди, в глаза селою ничи
И чконечем молоденком
Рышет погонищь верстой уры,
Ковсянойных лотных помлей,
Русская песню клесней...
Госпоясей, как песнь...
Но и зели, охозрачных,
Исясь, прашноствалился улыба
На тончине кольца.
За сегодня тебе, я не баю:
Просту....,
Обветлобнись с затуный рокой
Разбуди в целовек.

В прагодо
